In [22]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta, date
import concurrent.futures

In [21]:
DOMAIN = "http://news.ltn.com.tw"

dates = []
d_format = "%Y%m%d"
date = datetime.strptime("20140101", d_format)
delta = timedelta(days=1)
today = datetime.strptime(date.today().strftime(d_format), d_format)
while date <= today:
    dates.append(date.strftime("%Y%m%d"))
    date += delta

In [5]:
resp = requests.get("http://news.ltn.com.tw/list/newspaper/politics/20171101")
soup = BeautifulSoup(resp.text, "html5lib")
[a.get("href") for a in soup.select("ul.list > li > a.tit")]

['news/politics/paper/1148163',
 'news/politics/paper/1148164',
 'news/politics/paper/1148165',
 'news/politics/paper/1148166',
 'news/politics/paper/1148167',
 'news/politics/paper/1148168',
 'news/politics/paper/1148169',
 'news/politics/paper/1148170',
 'news/politics/paper/1148171',
 'news/politics/paper/1148172',
 'news/politics/paper/1148173',
 'news/politics/paper/1148174',
 'news/politics/paper/1148175',
 'news/politics/paper/1148176',
 'news/politics/paper/1148177',
 'news/politics/paper/1148178',
 'news/politics/paper/1148179',
 'news/politics/paper/1148180']

In [30]:
resp = requests.get("http://news.ltn.com.tw/news/politics/paper/1148163")
soup = BeautifulSoup(resp.text, "html5lib")
news = {}

news["title"]    = soup.select("div.articlebody > h1")[0].text.replace("\n", "").replace("\t", "")
news["datetime"] = soup.select("div.text > span")[0].text

content = "".join([p.text.strip() for p in soup.select("div.text > p")])
news["content"]  = re.findall("〔.+〕(.+)", content)[0]
news["hash"]     = hash(resp.text)
news["media"]    = "自由時報"
news["category"] = "politics"

In [31]:
news

{'category': 'politics',
 'content': '蔡英文總統正出訪南太三友邦，當地時間十月卅一日她在馬紹爾國會發表演說時當場宣布，在不久的將來，台灣即將給予包括馬紹爾在內的六個太平洋友邦國民一致的免簽證待遇，相信彼此的關係將更進一步提升。目前台灣給予美國、英國、澳洲等六十國的旅客免簽證，也將自十一月起至明年七月，試辦菲律賓國民來台十四天免簽證措施。蔡總統宣布六個太平洋友邦免簽，除馬紹爾外，還包括吐瓦魯、索羅門及另三個此行未到訪的吉里巴斯、帛琉和諾魯。至於免簽實施時間與待遇細節，外交部亞太司副司長藍夏禮昨說，將在總統訪問結束後，繼續研究、再適時對外宣布。目前我國給予南太六友邦的簽證待遇均為電子簽證三十天。蔡總統發表國會演說前，由當地學生獻歌歡迎，隨後演唱馬紹爾國歌，並由牧師帶領祈禱儀式。馬國國會議長凱迪致詞時說，他去年雙十期間訪問台灣，「我永遠不會忘記那次旅程，也視之為兩國友好的里程碑。」他竭誠歡迎蔡總統到訪，並表示這是首次有外國元首到馬紹爾國會發表演說。蔡總統指出，台灣和馬紹爾群島有很多相似的地方。同處於一片海洋上，也都走過歷史的艱難，經過幾個世代的努力，才蛻變成今天民主自由的國家。剛剛到這裡時，她寫下了「海洋國家，民主永續」八個字，這就是我們共同的連結。蔡總統說，去年八月，馬國國會通過「強化台灣及馬紹爾群島關係」的第二十八號決議案，這是馬國國會重視台馬邦交的具體行動。而總統海妮也以行動支持台灣參與「世界衛生大會」、「聯合國氣候變化綱要公約」及「國際民航組織」等重要的國際組織和公約，她要向各位表達台灣人民的由衷感謝。蔡總統強調，台灣推動「踏實外交、互惠互助」，就是希望和邦交國合作，加強彼此的實質關係，進而達到互惠互利的雙贏關係。並表示，堅持兼顧發展和生態永續是兩國合作重要的元素，在對抗氣候變遷的路上，馬紹爾群島永遠都有台灣這個重要的夥伴相互支持。總統也提到兩國間共通的南島文化，這次原民會主委及原住民族立委也隨團來訪，希望以南島文化為媒介，持續加深彼此間的交流與了解。在離開馬紹爾後，蔡總統十一月一日將前往吐瓦魯及索羅門群島訪問。',
 'datetime': '2017-11-01',
 'hash': -2644111247259128795,
 'media': '自由時報',
 'title': '蔡出訪送大禮 南太6友邦免簽'}

In [ ]:
def worker(date):
    resp = requests.get("http://news.ltn.com.tw/list/newspaper/politics/{}".format(date))
    soup = BeautifulSoup(resp.text, "html5lib")
    urls = [a.get("href") for a in soup.select("ul.list > li > a.tit")]
    
    for path in paths:
        url = DOMAIN + path
        c_resp = requests.get(url)
        soup = BeautifulSoup(c_resp.text, "html5lib")
        
        news = {}
        news["title"]    = soup.select("div.articlebody > h1")[0].text.replace("\n", "").replace("\t", "")
        news["datetime"] = soup.select("div.text > span")[0].text
        content = "".join([p.text.strip() for p in soup.select("div.text > p")])
        news["content"]  = re.findall("〔.+〕(.+)", content)[0]
        news["hash"]     = hash(c_resp.text)
        news["media"]    = "自由時報"
        news["category"] = "politics"
        news["url"] = url

In [ ]:
def main():
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        executor.map(worker, dates)

In [ ]:
if __name__ == "__main__":
    main()

datetime.datetime(2017, 11, 24, 0, 0)